In [ ]:
from IPython.display import display, clear_output

clear_output(wait=True)

import pandas as pd
from pymongo import MongoClient

%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.animation import FuncAnimation
import folium
from folium.features import DivIcon
from datetime import datetime


In [ ]:
# MongoDB setup
hostip = "192.168.0.160"
client = MongoClient(f"mongodb://{hostip}:27017")
violation_col = client["awas_db"]["violation"]
camera_col = client["awas_db"]["camera"]
latest_time = "0000-00-00T00:00:00"  # lowest ISO string

# get camera coordinates (latitude, longitude)
camera_coords = {}
for camera in camera_col.find({}):
    camera_coords[camera["_id"]] = (camera["latitude"], camera["longitude"])


In [ ]:
# plot setup
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
fig.tight_layout(pad=6)

fig.subplots_adjust(hspace=0.6, wspace=0.2)

# Initialize history DataFrame
history_df = pd.DataFrame(
    columns=["timestamp", "recorded_speed", "rounded_time"]
)


def annotate_max(x, y, ax):
    if not y:
        return
    ymax = max(y)
    xpos = y.index(ymax)
    xmax = x[xpos]
    text = f"Max: Time={xmax}, Value={ymax:.2f}"

    ax.annotate(
        text,
        xy=(xmax, ymax),
        xytext=(0, -20),  # 20 points below
        textcoords="offset points",
        arrowprops=dict(facecolor="red", arrowstyle="->", linewidth=1),
        fontsize=8,
    )


def annotate_min(x, y, ax):
    if not y:
        return
    ymin = min(y)
    xpos = y.index(ymin)
    xmin = x[xpos]
    text = f"Min: Time={xmin}, Value={ymin:.2f}"

    ax.annotate(
        text,
        xy=(xmin, ymin),
        xytext=(0, 20),  # 20 points above
        textcoords="offset points",
        arrowprops=dict(facecolor="orange", arrowstyle="->", linewidth=1),
        fontsize=8,
    )


def annotate_avg(x, y, ax):
    if not y:
        return
    avg_val = sum(y) / len(y)
    text = f"Avg: {avg_val:.2f}"

    ax.axhline(avg_val, color="blue", linestyle="--", linewidth=1)
    ax.annotate(
        text,
        xy=(x[-1], avg_val),
        xytext=(0, 10),  # Slightly above
        textcoords="offset points",
        arrowprops=dict(facecolor="blue", arrowstyle="->", linewidth=1),
        fontsize=8,
    )


def fetch_latest_data(since_time):
    """
    Fetches the latest violation data from MongoDB since the given time.

    @param since_time: ISO formatted string representing the time to fetch data from
    @return: DataFrame containing the latest violation data
    """
    pipeline = [
        {"$unwind": "$violations"},
        {
            "$project": {
                "timestamp": "$violations.timestamp_start",
                "recorded_speed": "$violations.recorded_speed",
            }
        },
        {"$match": {"timestamp": {"$gt": since_time}}},
    ]
    return pd.DataFrame(list(violation_col.aggregate(pipeline)))


def update(frame):
    """
    Update function for the animation. Fetches new data, processes it, and updates the plots.

    @param frame: Current frame number
    """
    global latest_time, history_df
    window_hours = 1

    # Fetch new data
    df = fetch_latest_data(latest_time)
    if df.empty:
        print("[Tick] No new data.")
        return

    # Process new data
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    df.dropna(subset=["timestamp"], inplace=True)
    df["rounded_time"] = df["timestamp"].dt.floor("min")
    latest_time = df["timestamp"].max().isoformat()

    # Append to history and prune by time window
    history_df = pd.concat([history_df, df], ignore_index=True)
    cutoff_time = pd.to_datetime(latest_time) - pd.Timedelta(
        hours=window_hours
    )
    history_df = history_df[history_df["timestamp"] >= cutoff_time]

    violation_df = (
        history_df.groupby("rounded_time")
        .size()
        .reset_index(name="num_violations")
    )
    speed_df = (
        history_df.groupby("rounded_time")["recorded_speed"]
        .mean()
        .reset_index(name="avg_speed")
    )

    # Clear previous plots
    ax1.cla()
    ax2.cla()

    # violations plot
    x1 = list(violation_df["rounded_time"])
    y1 = list(violation_df["num_violations"])
    ax1.plot(x1, y1, "b-o")
    ax1.set_title("Number of Violations vs Time")
    ax1.set_ylabel("Violations")
    ax1.set_xlabel("Time")
    annotate_max(x1, y1, ax1)
    annotate_min(x1, y1, ax1)
    annotate_avg(x1, y1, ax1)

    # speed plot
    x2 = list(speed_df["rounded_time"])
    y2 = list(speed_df["avg_speed"])
    ax2.plot(x2, y2, "g-o")
    ax2.set_title("Average Speed vs Time")
    ax2.set_ylabel("Speed (km/h)")
    ax2.set_xlabel("Time")
    annotate_max(x2, y2, ax2)
    annotate_min(x2, y2, ax2)
    annotate_avg(x2, y2, ax2)

    # x-axis limits
    if x1:
        ax1.set_xlim([cutoff_time, x1[-1]])
    if x2:
        ax2.set_xlim([cutoff_time, x2[-1]])

    # Formatting
    fmt = mdates.DateFormatter("%Y-%m-%d : %H:%M:%S")
    ax1.xaxis.set_major_formatter(fmt)
    ax2.xaxis.set_major_formatter(fmt)
    for label in ax1.get_xticklabels() + ax2.get_xticklabels():
        label.set_rotation(30)
        label.set_ha("right")
    ax1.tick_params(axis="x", labelsize=8)
    ax2.tick_params(axis="x", labelsize=8)

    fig.canvas.draw_idle()
    print(
        f"[Tick] Plotted {len(df)} new records. Total in window: {len(history_df)}"
    )


# === Animate ===
ani = FuncAnimation(fig, update, interval=2000)
plt.show()


In [ ]:
def annotate_violation_summary_and_hotspots(
    fmap,
    violation_col,
    camera_coords,
    start_time,
    end_time,
    threshold=50,
    offset_meters=100,
):
    """
    Annotates the map with violation summaries and hotspots.

    @param fmap: Folium map object to annotate
    @param violation_col: MongoDB collection containing violation data
    @param camera_coords: Dictionary mapping camera IDs to their coordinates
    @param start_time: Start time for filtering violations
    @param end_time: End time for filtering violations
    @param threshold: Minimum number of violations to consider a hotspot
    @param offset_meters: Offset in meters to adjust marker positions
    @return: Annotated Folium map object
    """

    # Helper functions
    def midpoint(coord1, coord2):
        return [(coord1[0] + coord2[0]) / 2, (coord1[1] + coord2[1]) / 2]

    def offset(coord, offset_m=10):
        # Approximate conversion: 1 degree lat ≈ 111,000 meters
        offset_deg = offset_m / 111000
        return [coord[0] + offset_deg, coord[1] + offset_deg]

    pipeline = [
        {"$unwind": "$violations"},
        {
            "$project": {
                "car_plate": 1,
                "camera_id_start": "$violations.camera_id_start",
                "camera_id_end": "$violations.camera_id_end",
                "timestamp_start": "$violations.timestamp_start",
                "timestamp_end": "$violations.timestamp_end",
                "type": "$violations.type",
            }
        },
        {
            "$match": {
                "timestamp_start": {
                    "$gte": start_time.isoformat(),
                    "$lt": end_time.isoformat(),
                }
            }
        },
    ]

    df = pd.DataFrame(list(violation_col.aggregate(pipeline)))

    df_instant = df[df["type"] == "instant"]
    df_average = df[df["type"] == "average"]

    instant_counts = df_instant.groupby("camera_id_start").size().to_dict()

    avg_pairs = (
        df_average.groupby(["camera_id_start", "camera_id_end"])
        .size()
        .reset_index(name="count")
    )

    average_counts = {
        (row["camera_id_start"], row["camera_id_end"]): row["count"]
        for _, row in avg_pairs.iterrows()
    }

    for cam_id, coord in camera_coords.items():
        count = instant_counts.get(cam_id, 0)
        offset_coord = offset(coord, offset_meters)
        folium.Marker(
            location=offset_coord,
            icon=DivIcon(
                icon_size=(150, 36),
                icon_anchor=(0, 0),
                html=f'<div style="font-size: 12px; color: red;"><b>Instant: {count}</b></div>',
            ),
        ).add_to(fmap)

        # Add hotspot if above threshold
        if count >= threshold:
            folium.CircleMarker(
                location=coord,
                radius=20,
                color="red",
                fill=True,
                fill_opacity=0.5,
                popup=f"‼️🚨 Hotspot: {count} violations occurred!",
            ).add_to(fmap)

    for (cam1, cam2), count in average_counts.items():
        if cam1 in camera_coords and cam2 in camera_coords:
            mid = midpoint(camera_coords[cam1], camera_coords[cam2])
            offset_mid = offset(mid, offset_meters)

            folium.Marker(
                location=offset_mid,
                icon=DivIcon(
                    icon_size=(150, 36),
                    icon_anchor=(0, 0),
                    html=f'<div style="font-size: 12px; color: orange;"><b>Avg {cam1}→{cam2}: {count}</b></div>',
                ),
            ).add_to(fmap)

            if count >= threshold:
                folium.CircleMarker(
                    location=mid,
                    radius=15,
                    color="orange",
                    fill=True,
                    fill_opacity=0.5,
                    popup=f"‼️🚨 Hotspot: {count} violations occurred!",
                ).add_to(fmap)

    for loc in camera_coords.values():
        folium.Marker(
            location=loc, popup=f"lat={loc[0]:.2f}, lon={loc[1]:.2f}"
        ).add_to(fmap)

    return fmap


fomap = folium.Map(location=[2.162418757, 102.6524549], zoom_start=15)

start_time = datetime.fromisoformat("2024-01-01T08:00:00")
end_time = datetime.fromisoformat("2024-01-01T21:30:00")

fomap = annotate_violation_summary_and_hotspots(
    fmap=fomap,
    violation_col=violation_col,
    camera_coords=camera_coords,
    start_time=start_time,
    end_time=end_time,
    threshold=500,
    offset_meters=100,
)

fomap


#### Explanation & Justification
This plotted map shows the latitude and longitude of the 3 cameras, with camera 2's latitude and longitude being the initialisation of the map to quickly identify the positions of the cameras. Each camera position is annotated with a blue Marker, with the instantaneous number of violations labelled in red beside each of them. The number of violations in between each pairs of cameras are also labelled showing the average violations happened along the path. 

For hotspot identification, we have set the timestamp on 2024/01/01 at 8:00am to 2024/01/01 9:30pm to display morning to evening assuming we are querying from a populated violation database. Based on this timeframe, we have annotated the hotspots using threshold of 500, indicating the positions with number of violations that exceed 500 within this timeframe will be identified as hotspots. The threshold is set according to the testing data where we observed the whole day data appears to have large amount of violations in our selected timeframe, hence a 500 threshold is reasonably selected to represent the high violation occurence. These hotspots are annotated with yellow and red CircleMarker for average and instantaneous violations respectively. Average speed violations are plotted in between cameras while instantaneous is plotted right under each camera.

The timeframe and threshold are parameterizable so that we can perform customised query using different definitions of hotspot threshold during different timeframes to perform data analysis.